### add rsid mapping

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
chrom=pd.read_csv("/cellar/users/mpagadal/data/dbsnp/grch38.chrom.mapping.txt",header=None)
chrom=chrom.loc[0:23]
chrom[1]=chrom[0].str.split(".").str[0].str.split("_").str[1].astype(int)
mp_chrom=dict(zip(chrom[1],chrom[0]))

In [3]:
trans=pd.read_csv("../supplemental/tables1.csv",index_col=0)
ancestry=pd.read_csv("../supplemental/tables2.csv",index_col=0)

In [4]:
df_snps=pd.DataFrame({"snps":trans["SNP"].tolist()+ancestry["SNP"].tolist()})

In [5]:
df_snps=pd.DataFrame({"snps":trans["SNP"].tolist()+ancestry["SNP"].tolist()})
df_snps["snps"]=df_snps["snps"].str.replace("chr","")
df_snps["chr"]=df_snps["snps"].str.split(":").str[0]
df_snps["chr"]=df_snps["chr"].replace("X",23)
df_snps["chr"]=df_snps["chr"].astype(int)
df_snps["bp1"]=df_snps["snps"].str.split(":").str[1]
df_snps["bp2"]=df_snps["bp1"].astype(int)+1
df_snps["bp1"]=df_snps["bp1"].astype(int)-1
df_snps["chr_id"]=df_snps["chr"].map(mp_chrom)
df_snps[["chr_id","bp1","bp2"]].to_csv("map.bed",header=None,index=None,sep="\t")

In [6]:
chrom=pd.read_csv("/cellar/users/mpagadal/data/dbsnp/grch38.chrom.mapping.txt",header=None)
chrom=chrom.loc[0:23]
chrom[1]=chrom[0].str.split(".").str[0].str.split("_").str[1].astype(int)
mp_chrom=dict(zip(chrom[0],chrom[1]))

In [7]:
rsid=pd.read_csv("/cellar/users/mpagadal/data/dbsnp/map.dbsnp.txt",comment="#",delimiter="\t",header=None)
rsid["chr"]=rsid[0].map(mp_chrom)
rsid["chr"]=rsid["chr"].replace(23,"X")
rsid["ID"]=rsid["chr"].astype(str)+":"+rsid[1].astype(str)
mp_rsid=dict(zip(rsid["ID"],rsid[2]))

In [8]:
trans["ID"]=trans["SNP"].str.rsplit(":",2).str[0]
trans["rsid"]=trans["ID"].map(mp_rsid)

In [9]:
ancestry["ID"]=ancestry["SNP"].str.rsplit(":",2).str[0]
ancestry["ID"]=ancestry["ID"].str.replace("chr","")
ancestry["rsid"]=ancestry["ID"].map(mp_rsid)

In [10]:
ancestry[ancestry["rsid"].isnull()]

,F,SNP,BP,P,pheno,group,A1,BETA,SE,consequence,gene,missense,ID,specificity,all_sig,susie,ancestry_beta,rsid
CHR,,,,,,,,,,,,,,,,,,


In [11]:
trans.to_csv("../supplemental/tables1.csv",index=None,sep="\t")
ancestry.to_csv("../supplemental/tables2.csv",index=None,sep="\t")

In [29]:
trans=pd.read_csv("../supplemental/tables1.csv",delimiter="\t")

### make MR input

In [41]:
direct="../data/summarystats/extract/"
compiled_stats=pd.DataFrame()
for x in [x for x in os.listdir(direct)]:
    df=pd.read_csv(direct+x,delimiter="\t")
    df["file"]=x
    df["pheno"]=x.split(".")[2]
    df["group"]=x.split(".")[1]
    if x.split(".")[2]=="hypogonad":
        df["BETA"]=np.log(df["OR"])
        df["SE"]=df["LOG(OR)_SE"]
    compiled_stats=compiled_stats.append(df)
compiled_stats["SNP"]=compiled_stats["ID"].str.split("chr").str[1]

In [54]:
for group in ["EUR","AFR","AMR","EAS"]:
    check=compiled_stats[compiled_stats["group"]==group]
    df=pd.merge(trans[["SNP","pheno","rsid"]],check,on=["SNP","pheno"],how="left")[["rsid","BETA","SE","A1","P","pheno"]]
    for pheno in df["pheno"].unique():
        df[df["pheno"]==pheno][["rsid","BETA","SE","A1","P"]].to_csv(group+"."+pheno+".sumstats.forMR.csv",index=None)

In [49]:
pd.merge(trans[["SNP","pheno","rsid"]],check,on=["SNP","pheno"],how="left")[["rsid","BETA","SE","A1","P","pheno"]]

,rsid,BETA,SE,A1,P,pheno
0,rs79598313,0.148394,0.030287,T,9.600480e-07,hypogonad
1,rs7893850,0.063472,0.012470,G,3.579260e-07,hypogonad
2,rs10822160,-0.136707,0.008898,G,2.883940e-53,hypogonad
3,rs10996471,-0.035267,0.009027,G,9.355360e-05,hypogonad
4,rs56196860,-0.312310,0.025923,A,2.002290e-33,hypogonad
...,...,...,...,...,...,...
159,rs202200760,0.167764,0.025391,C,4.059220e-11,shbg
160,rs1260326,-0.068539,0.011136,T,7.721380e-10,shbg
161,rs738408,0.065583,0.013119,T,5.830840e-07,shbg
162,rs1154988,0.057360,0.013302,T,1.628760e-05,shbg


In [70]:
compiled_stats[(compiled_stats["group"]=="EAS")&(compiled_stats["pheno"]=="hypogonad")]

,#CHROM,POS,ID,REF,ALT,A1,TEST,OBS_CT,BETA,SE,...,P,ERRCODE,file,pheno,group,FIRTH?,OR,LOG(OR)_SE,Z_STAT,SNP
0,1,26958422,chr1:26958422:C:T,C,T,T,ADD,1690,1.312115,0.694250,...,0.058761,.,compiled.EAS.hypogonad.v2.extract.testosterone...,hypogonad,EAS,N,3.714020,0.694250,1.889980,1:26958422:C:T
1,10,5209519,chr10:5209519:A:G,A,G,G,ADD,1690,0.281518,0.099100,...,0.004501,.,compiled.EAS.hypogonad.v2.extract.testosterone...,hypogonad,EAS,N,1.325140,0.099100,2.840700,10:5209519:A:G
2,10,5270402,chr10:5270402:G:A,G,A,A,ADD,1690,0.155943,0.111521,...,0.162007,.,compiled.EAS.hypogonad.v2.extract.testosterone...,hypogonad,EAS,N,1.168760,0.111521,1.398350,10:5270402:G:A
3,10,63353036,chr10:63353036:T:G,T,G,G,ADD,1690,-0.126862,0.076950,...,0.099223,.,compiled.EAS.hypogonad.v2.extract.testosterone...,hypogonad,EAS,N,0.880855,0.076950,-1.648630,10:63353036:T:G
4,10,63376868,chr10:63376868:C:T,C,T,C,ADD,1690,0.108926,0.074193,...,0.142067,.,compiled.EAS.hypogonad.v2.extract.testosterone...,hypogonad,EAS,N,1.115080,0.074193,1.468140,10:63376868:C:T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,X,68183881,chrX:68183881:T:G,T,G,G,ADD,1690,-0.088094,0.079857,...,0.269967,.,compiled.EAS.hypogonad.v2.extract.testosterone...,hypogonad,EAS,Y,0.915675,0.079857,-1.103140,X:68183881:T:G
115,X,68264549,chrX:68264549:A:G,A,G,G,ADD,1690,-0.079358,0.079097,...,0.315713,.,compiled.EAS.hypogonad.v2.extract.testosterone...,hypogonad,EAS,Y,0.923709,0.079097,-1.003310,X:68264549:A:G
116,X,110440911,chrX:110440911:G:C,G,C,C,ADD,1690,-0.050238,0.110466,...,0.649269,.,compiled.EAS.hypogonad.v2.extract.testosterone...,hypogonad,EAS,Y,0.951003,0.110466,-0.454778,X:110440911:G:C
117,X,110445924,chrX:110445924:A:G,A,G,G,ADD,1690,-0.048888,0.110489,...,0.658149,.,compiled.EAS.hypogonad.v2.extract.testosterone...,hypogonad,EAS,Y,0.952288,0.110489,-0.442470,X:110445924:A:G


In [38]:
for pheno in trans["pheno"].unique():
    trans[trans["pheno"]==pheno][["rsid","BETA","SE","A1","P"]].to_csv(pheno+".sumstats.forMR.csv",index=None)

In [13]:
trans_nox=trans[trans["CHR"]!=23]

In [14]:
for pheno in trans_nox["pheno"].unique():
    trans_nox[trans_nox["pheno"]==pheno][["rsid","BETA","SE","A1","P"]].to_csv(pheno+".sumstats.forMR.noX.csv",index=None)